<a href="https://colab.research.google.com/github/frukkurt/Honest_data_test/blob/main/Honest_pinyawat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA IMPORT

In [ ]:
!pip install kaggle
!mkdir kaggle
!git clone https://github.com/frukkurt/Honest_data_test
!cp /content/Honest_data_test/kaggle.json  /content/kaggle
!chmod 600 /content/kaggle/kaggle.json


Cloning into 'Honest_data_test'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [ ]:
%cd /content/kaggle

/content/kaggle


In [ ]:
!kaggle competitions download -c competitive-data-science-predict-future-sales
!unzip /content/kaggle/competitive-data-science-predict-future-sales.zip

competitive-data-science-predict-future-sales.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/kaggle/competitive-data-science-predict-future-sales.zip
replace item_categories.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: item_categories.csv     
replace items.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace items.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: items.csv               
replace sales_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sales_train.csv         
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace shops.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: shops.csv               
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                


In [ ]:
import pandas as pd

sales_train=pd.read_csv("sales_train.csv")
item_categories=pd.read_csv("item_categories.csv")
sample_submission=pd.read_csv("sample_submission.csv")
items=pd.read_csv("items.csv")
shops=pd.read_csv("shops.csv")
test=pd.read_csv("test.csv")

# LIB setup


In [ ]:
import pandas as pd

from tqdm import tqdm
import numpy as np

import pickle as p

import tensorflow as tf
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder



import plotly
import plotly.graph_objs as go
import plotly.express as px



#EDA and OBJECTIVE

***Objective***: for predicted time series for each **shop_id** and **item_id**

`File descriptions`

*   sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.

*   test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.

*   sample_submission.csv - a sample submission file in the correct format.

*   items.csv - supplemental information about the items/products.

*   item_categories.csv  - supplemental information about the items categories.

*   shops.csv- supplemental information about the shops.


`Data fields`

*   ID - an Id that represents a (Shop, Item) tuple within the test set

*   shop_id - unique identifier of a shop

*   item_id - unique identifier of a product

*   item_category_id - unique identifier of item category

*   item_cnt_day - number of products sold. You are predicting a monthly amount of this measure

*   item_price - current price of an item

*   date - date in format dd/mm/yyyy

*   date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33

*   item_name - name of item

*   shop_name - name of shop

*   item_category_name - name of item category

sales_train

In [ ]:
display(sales_train)
print("---info---")
display(sales_train.info())
print("---Check missing value---")
display(sales_train.isnull().sum())
print("---Check nan---")
display(sales_train.isna().sum())
print("---Describe---")
display(sales_train.describe())

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


---info---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


None

---Check missing value---


date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

---Check nan---


date              0
date_block_num    0
shop_id           0
item_id           0
item_price        0
item_cnt_day      0
dtype: int64

---Describe---


,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03


In [ ]:
print("SHOP_ID")
print(sales_train.shop_id.unique())
print("Number:",len(sales_train.shop_id.unique()))

print()
print("ITEM_ID")
print(sales_train.item_id.unique())
print("Number:",len(sales_train.item_id.unique()))



SHOP_ID
[59 25 24 23 19 22 18 21 28 27 29 26  4  6  2  3  7  0  1 16 15  8 10 14
 13 12 53 31 30 32 35 56 54 47 50 42 43 52 51 41 38 44 37 46 45  5 57 58
 55 17  9 49 39 40 48 34 33 20 11 36]
Number: 60

ITEM_ID
[22154  2552  2554 ...  7610  7635  7640]
Number: 21807


item_categories

In [ ]:
display(item_categories)
print("---info---")
display(item_categories.info())
print("---Check missing value---")
display(item_categories.isnull().sum())
print("---Check nan---")
display(item_categories.isna().sum())


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4
...,...,...
79,Служебные,79
80,Служебные - Билеты,80
81,Чистые носители (шпиль),81
82,Чистые носители (штучные),82


---info---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   item_category_name  84 non-null     object
 1   item_category_id    84 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.4+ KB


None

---Check missing value---


item_category_name    0
item_category_id      0
dtype: int64

---Check nan---


item_category_name    0
item_category_id      0
dtype: int64

items

In [ ]:
display(items)
print("---info---")
display(items.info())
print("---Check missing value---")
display(items.isnull().sum())
print("---Check nan---")
display(items.isna().sum())

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40
...,...,...,...
22165,"Ядерный титбит 2 [PC, Цифровая версия]",22165,31
22166,Язык запросов 1С:Предприятия [Цифровая версия],22166,54
22167,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,22167,49
22168,Яйцо для Little Inu,22168,62


---info---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22170 entries, 0 to 22169
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   item_name         22170 non-null  object
 1   item_id           22170 non-null  int64 
 2   item_category_id  22170 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 519.7+ KB


None

---Check missing value---


item_name           0
item_id             0
item_category_id    0
dtype: int64

---Check nan---


item_name           0
item_id             0
item_category_id    0
dtype: int64

shops

In [ ]:
display(shops)
print("---info---")
display(shops.info())
print("---Check missing value---")
display(shops.isnull().sum())
print("---Check nan---")
display(shops.isna().sum())

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4
5,"Вологда ТРЦ ""Мармелад""",5
6,"Воронеж (Плехановская, 13)",6
7,"Воронеж ТРЦ ""Максимир""",7
8,"Воронеж ТРЦ Сити-Парк ""Град""",8
9,Выездная Торговля,9


---info---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   shop_name  60 non-null     object
 1   shop_id    60 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


None

---Check missing value---


shop_name    0
shop_id      0
dtype: int64

---Check nan---


shop_name    0
shop_id      0
dtype: int64

Explore Shop_id and item_id

notice: some shop and item appear just 1 is mean in those stores Able to sell that product only 1 time from the data collection period.


In [ ]:
st=sales_train.copy()
st["combine_id"]=[str(i)+"_"+str(j) for (i,j) in zip(st.shop_id,st.item_id)]
st.combine_id.value_counts()

28_20949    867
53_20949    863
31_20949    863
22_20949    848
42_20949    834
           ... 
16_14635      1
35_11001      1
35_11045      1
27_11937      1
59_22154      1
Name: combine_id, Length: 424124, dtype: int64

Explore Item in term **item_price**	and item_cnt_day

In [ ]:
#best frequency item
in_item=20949
in_shop=28
best_=sales_train[(sales_train.shop_id==in_shop) & (sales_train.item_id==in_item)]
best_=best_.sort_values(by="date")

#change data to date time
best_['Datetime']=pd.to_datetime(best_['date'], format="%d.%m.%Y")

display(best_)

fig_trends = px.scatter(best_, x="Datetime", y="item_cnt_day",trendline="ols",title="Trends of sales unit in item {} and shop {}".format(in_item,in_shop))
fig_trends.show()


fig_prices = px.scatter(best_, x="Datetime", y="item_price",trendline="ols",title="Trends of prices in item {} and shop {}".format(in_item,in_shop))
fig_prices.show()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,Datetime
1307531,01.01.2014,12,28,20949,5.0,40.0,2014-01-01
2373832,01.01.2015,24,28,20949,5.0,37.0,2015-01-01
1389001,01.02.2014,13,28,20949,5.0,35.0,2014-02-01
2467299,01.02.2015,25,28,20949,5.0,21.0,2015-02-01
1514931,01.03.2014,14,28,20949,5.0,23.0,2014-03-01
...,...,...,...,...,...,...,...
958575,31.10.2013,9,28,20949,5.0,12.0,2013-10-31
2028582,31.10.2014,21,28,20949,5.0,14.0,2014-10-31
2928474,31.10.2015,33,28,20949,5.0,15.0,2015-10-31
1146302,31.12.2013,11,28,20949,5.0,35.0,2013-12-31


In [ ]:
#best frequency item
in_item=20949
in_shop=53
best_=sales_train[(sales_train.shop_id==in_shop) & (sales_train.item_id==in_item)]
best_=best_.sort_values(by="date")

#change data to date time
best_['Datetime']=pd.to_datetime(best_['date'], format="%d.%m.%Y")

display(best_)



fig_trends = px.scatter(best_, x="Datetime", y="item_cnt_day",trendline="ols",title="Trends of sales unit in item {} and shop {}".format(in_item,in_shop))
fig_trends.show()


fig_prices = px.scatter(best_, x="Datetime", y="item_price",trendline="ols",title="Trends of prices in item {} and shop {}".format(in_item,in_shop))
fig_prices.show()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,Datetime
1426765,01.02.2014,13,53,20949,5.0,5.0,2014-02-01
2445600,01.02.2015,25,53,20949,5.0,1.0,2015-02-01
1461828,01.03.2014,14,53,20949,5.0,11.0,2014-03-01
2517520,01.03.2015,26,53,20949,5.0,4.0,2015-03-01
1553875,01.04.2014,15,53,20949,5.0,4.0,2014-04-01
...,...,...,...,...,...,...,...
975119,31.10.2013,9,53,20949,5.0,9.0,2013-10-31
2085562,31.10.2014,21,53,20949,5.0,6.0,2014-10-31
2905848,31.10.2015,33,53,20949,5.0,4.0,2015-10-31
1199679,31.12.2013,11,53,20949,5.0,21.0,2013-12-31


In [ ]:
#best frequency item
in_item=20949
in_shop=31
best_=sales_train[(sales_train.shop_id==in_shop) & (sales_train.item_id==in_item)]
best_=best_.sort_values(by="date")

#change data to date time
best_['Datetime']=pd.to_datetime(best_['date'], format="%d.%m.%Y")

display(best_)



fig_trends = px.scatter(best_, x="Datetime", y="item_cnt_day",trendline="ols",title="Trends of sales unit in item {} and shop {}".format(in_item,in_shop))
fig_trends.show()


fig_prices = px.scatter(best_, x="Datetime", y="item_price",trendline="ols",title="Trends of prices in item {} and shop {}".format(in_item,in_shop))
fig_prices.show()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,Datetime
1385971,01.02.2014,13,31,20949,5.0,30.0,2014-02-01
2421506,01.02.2015,25,31,20949,5.0,15.0,2015-02-01
1520527,01.03.2014,14,31,20949,5.0,45.0,2014-03-01
1625412,01.04.2014,15,31,20949,5.0,9.0,2014-04-01
484470,01.05.2013,4,31,20949,5.0,1.0,2013-05-01
...,...,...,...,...,...,...,...
952311,31.10.2013,9,31,20949,5.0,44.0,2013-10-31
2094003,31.10.2014,21,31,20949,5.0,23.0,2014-10-31
2888826,31.10.2015,33,31,20949,5.0,16.0,2015-10-31
1234735,31.12.2013,11,31,20949,5.0,74.0,2013-12-31


From visual trend of unit is decrease but price quite stable

*Therefore, the price was not taken into account in order to keep up with the problem for 2-4 hours.

# Data Prep

Group combination of Shop_id and Item_id

In [ ]:
shop_item_com_id=sales_train.groupby(['shop_id', 'item_id'])[['item_price']].mean().reset_index()
shop_item_com_id=shop_item_com_id[['shop_id', 'item_id']]
shop_item_com_id

,shop_id,item_id
0,0,30
1,0,31
2,0,32
3,0,33
4,0,35
...,...,...
424119,59,22154
424120,59,22155
424121,59,22162
424122,59,22164


Find average sale unit by date_block_num for shop_id and item_id

In [ ]:
gb_df=sales_train.groupby(["date_block_num",'shop_id','item_id'])[['item_cnt_day']].mean()
gb_df_=gb_df.reset_index()
gb_df_

,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,1.5
1,0,0,33,1.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,1.0
...,...,...,...,...
1609119,33,59,22087,2.0
1609120,33,59,22088,1.0
1609121,33,59,22091,1.0
1609122,33,59,22100,1.0


Create DataFrame for shop_id and item_id in term date_block(month interest)

- Used mean for unit sale per month

In [ ]:

block_num=list(set(sales_train.date_block_num.unique()))

for i in tqdm(block_num):
    if i ==0:
        gb_0=gb_df_[gb_df_.date_block_num==0]
        gb_0=gb_0[["shop_id","item_id","item_cnt_day"]]
        
        test_0=pd.merge(shop_item_com_id, gb_0, how="left",on=['shop_id', 'item_id'])
        df_main=test_0.rename(columns = {'item_cnt_day':'date_block_0'})
        
    else:
        gb_i=gb_df_[gb_df_.date_block_num==i]
        gb_i=gb_i[["shop_id","item_id","item_cnt_day"]]
        
        df_main=pd.merge(df_main, gb_i, how="left",on=['shop_id', 'item_id'])
        rename_='date_block_'+str(i)
        df_main=df_main.rename(columns = {'item_cnt_day':rename_})
        
#roundup        
df_main=df_main.apply(np.ceil) 
#Fill nan because some item can sell 1 time in Month
df_main=df_main.fillna(0)
df_main

100%|██████████| 34/34 [00:05<00:00,  6.15it/s]


,shop_id,item_id,date_block_0,date_block_1,date_block_2,date_block_3,date_block_4,date_block_5,date_block_6,date_block_7,...,date_block_24,date_block_25,date_block_26,date_block_27,date_block_28,date_block_29,date_block_30,date_block_31,date_block_32,date_block_33
0,0.0,30.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,31.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,32.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,33.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,35.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,59.0,22154.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
424120,59.0,22155.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
424121,59.0,22162.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
424122,59.0,22164.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
df_main=df_main.astype('int32')
df_main

,shop_id,item_id,date_block_0,date_block_1,date_block_2,date_block_3,date_block_4,date_block_5,date_block_6,date_block_7,...,date_block_24,date_block_25,date_block_26,date_block_27,date_block_28,date_block_29,date_block_30,date_block_31,date_block_32,date_block_33
0,0,30,0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,31,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,32,2,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,33,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,35,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424119,59,22154,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
424120,59,22155,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
424121,59,22162,0,0,0,0,0,0,0,0,...,0,1,1,1,1,0,0,1,0,0
424122,59,22164,0,0,0,0,0,0,0,0,...,0,1,1,1,0,0,1,0,0,0


Splite date in term Test and Train

*Training set*
*   x_train: sales unit data from `date_block_0` to `date_block_31`
*   y_train: sales unit data in month `date_block_32`

*Testing set*
*   x_test: sales unit data from `date_block_1` to `date_block_32`
*   y_test: sales unit data in month `date_block_33`

In [ ]:
x_train_df=df_main[df_main.columns[:-2]]
y_train_df=df_main[["shop_id",'item_id','date_block_32']]
x_test_df=df_main[df_main.columns[0:2].to_list()+df_main.columns[3:-1].to_list()]
y_test_df=df_main[["shop_id",'item_id','date_block_33']]

Label encode dor norminal category (do or not do)

In [ ]:
#Label encode
le = LabelEncoder()
le.fit(x_train_df.item_id)
x_label_encode=le.transform(x_train_df.item_id)
y_label_encode=le.transform(y_train_df.item_id)

x_train_df.item_id=x_label_encode
y_train_df.item_id=x_label_encode

x_test_df.item_id=x_label_encode
y_test_df.item_id=x_label_encode

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



One hot encode for shop id

In [ ]:
#One hot encode
dummy_x_train=pd.get_dummies(x_train_df.shop_id, prefix='Shop')
x_train_df_=pd.concat([dummy_x_train, x_train_df[x_train_df.columns[1:]]], axis=1)

dummy_x_test=pd.get_dummies(x_test_df.shop_id, prefix='Shop')
x_test_df_=pd.concat([dummy_x_test, x_test_df[x_test_df.columns[1:]]], axis=1)

In [ ]:
# get values for model
x_train=np.expand_dims(x_train_df_.values,axis = 2)

x_test=np.expand_dims(x_test_df_.values,axis = 2)

y_train=y_train_df[['date_block_32']].values
y_test=y_test_df[['date_block_33']].values

In [ ]:
# #DROP shop_id and item_id
# x_train=x_train_df[x_train_df.columns[2:]]
# x_train=np.expand_dims(x_train.values,axis = 2)

# x_test=x_test_df[x_test_df.columns[2:]]
# x_test=np.expand_dims(x_test.values,axis = 2)

# y_train=y_train_df[['date_block_32']].values
# y_test=y_test_df[['date_block_33']].values

In [ ]:
#Shape for model input
print("x_train_shape",x_train.shape)
print("x_test_shape",x_test.shape)
print("y_train_shape",y_train.shape)
print("y_test_shape",y_test.shape)

x_train_shape (424124, 93, 1)
x_test_shape (424124, 93, 1)
y_train_shape (424124, 1)
y_test_shape (424124, 1)


# MODEL

Since the dataset is a time series, `LSTM` type deep learning is used.

Create Model structure 

In [138]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(100,input_shape=[None,1],return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.LSTM(100))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(1,activation='relu'))



Use 
- loss: `mse` 
- optimizer : `adam`
- metrics : `rmse` ***from question***

In [139]:

model.compile(loss = 'mse',optimizer = 'adam', metrics = tf.keras.metrics.RootMeanSquaredError(name='rmse'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_5 (LSTM)               (None, None, 100)         40800     
                                                                 
 dropout_4 (Dropout)         (None, None, 100)         0         
                                                                 
 lstm_6 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________


***Training***

For save time choose batch_size=200 and epochs=10


In [140]:
his=model.fit(x_train,y_train,batch_size=200,epochs=10,verbose=1,validation_data=(x_test,y_test))

Epoch 1/10
2121/2121 [==============================] - 56s 25ms/step - loss: 0.1144 - rmse: 0.3382 - val_loss: 0.7465 - val_rmse: 0.8640
Epoch 2/10
2121/2121 [==============================] - 51s 24ms/step - loss: 0.1119 - rmse: 0.3345 - val_loss: 0.7468 - val_rmse: 0.8642
Epoch 3/10
2121/2121 [==============================] - 51s 24ms/step - loss: 0.1110 - rmse: 0.3332 - val_loss: 0.7438 - val_rmse: 0.8624
Epoch 4/10
2121/2121 [==============================] - 52s 24ms/step - loss: 0.1106 - rmse: 0.3326 - val_loss: 0.7388 - val_rmse: 0.8596
Epoch 5/10
2121/2121 [==============================] - 51s 24ms/step - loss: 0.1059 - rmse: 0.3255 - val_loss: 0.7375 - val_rmse: 0.8588
Epoch 6/10
2121/2121 [==============================] - 52s 25ms/step - loss: 0.1026 - rmse: 0.3203 - val_loss: 0.7374 - val_rmse: 0.8587
Epoch 7/10
2121/2121 [==============================] - 52s 24ms/step - loss: 0.1098 - rmse: 0.3313 - val_loss: 0.7485 - val_rmse: 0.8652
Epoch 8/10
2121/2121 [============

Result from model

ฑecommend: Result will be displayed as a `decimal` ,We should round the value. for example, if buying just 0.2 items should be rounded to 0 or 0.8 should be rounded to 1.

In [166]:

result=model.predict(x_test)

print("No round values")
print(mean_squared_error(y_test, result, squared=False))

print("Round values")
mean_squared_error(y_test, np.round(result), squared=False)




No round values
0.8687024239975489
Round values


0.8756855988778027

In [143]:

h1 = go.Scatter(y=his.history['loss'], 
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="loss"
                   )
h2 = go.Scatter(y=his.history['val_loss'], 
                    mode="lines", line=dict(
                    width=2,
                    color='red'),
                    name="val_loss"
                   )
data = [h1,h2]
layout1 = go.Layout(title='Loss',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig1 = go.Figure(data = data, layout=layout1)
fig1.show()

In [144]:
h3 = go.Scatter(y=his.history['rmse'], 
                    mode="lines", line=dict(
                    width=2,
                    color='blue'),
                    name="rmse"
                   )
h4 = go.Scatter(y=his.history['val_rmse'], 
                    mode="lines", line=dict(
                    width=2,
                    color='red'),
                    name="val_rmse"
                   )
data2 = [h3,h4]
layout2 = go.Layout(title='Rmse',
                   xaxis=dict(title='epochs'),
                   yaxis=dict(title=''))
fig2 = go.Figure(data = data2, layout=layout2)
fig2.show()

# model 2

Add dense before and after first LSTM layer 

In [168]:
from keras.layers import TimeDistributed

model_2 = tf.keras.Sequential()
model_2.add(tf.keras.layers.Dense(100,input_shape=[None,1],activation='relu'))
model_2.add(tf.keras.layers.LSTM(100,return_sequences=True))
model_2.add(tf.keras.layers.Dense(100,activation='relu'))
model_2.add(tf.keras.layers.Dropout(0.2))
model_2.add(tf.keras.layers.LSTM(100))
model_2.add(tf.keras.layers.Dropout(0.2))
model_2.add(tf.keras.layers.Dense(50,activation='relu'))
model_2.add(tf.keras.layers.Dense(1,activation='relu'))


model_2.compile(loss = 'mse',optimizer = 'adam', metrics = tf.keras.metrics.RootMeanSquaredError(name='rmse'))
model_2.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, None, 100)         200       
                                                                 
 lstm_13 (LSTM)              (None, None, 100)         80400     
                                                                 
 dense_14 (Dense)            (None, None, 100)         10100     
                                                                 
 dropout_12 (Dropout)        (None, None, 100)         0         
                                                                 
 lstm_14 (LSTM)              (None, 100)               80400     
                                                                 
 dropout_13 (Dropout)        (None, 100)               0         
                                                                 
 dense_15 (Dense)            (None, 50)              

In [169]:
his_2=model_2.fit(x_train,y_train,batch_size=200,epochs=10,verbose=1,validation_data=(x_test,y_test))


Epoch 1/10
2121/2121 [==============================] - 65s 29ms/step - loss: 0.1143 - rmse: 0.3381 - val_loss: 0.7434 - val_rmse: 0.8622
Epoch 2/10
2121/2121 [==============================] - 61s 29ms/step - loss: 0.1094 - rmse: 0.3308 - val_loss: 0.7402 - val_rmse: 0.8604
Epoch 3/10
2121/2121 [==============================] - 60s 29ms/step - loss: 0.1079 - rmse: 0.3285 - val_loss: 0.7449 - val_rmse: 0.8631
Epoch 4/10
2121/2121 [==============================] - 60s 28ms/step - loss: 0.1043 - rmse: 0.3230 - val_loss: 0.7358 - val_rmse: 0.8578
Epoch 5/10
2121/2121 [==============================] - 60s 28ms/step - loss: 0.1072 - rmse: 0.3273 - val_loss: 0.7386 - val_rmse: 0.8594
Epoch 6/10
2121/2121 [==============================] - 64s 30ms/step - loss: 0.1048 - rmse: 0.3238 - val_loss: 0.7352 - val_rmse: 0.8575
Epoch 7/10
2121/2121 [==============================] - 61s 29ms/step - loss: 0.1060 - rmse: 0.3255 - val_loss: 0.7427 - val_rmse: 0.8618
Epoch 8/10
2121/2121 [============

In [170]:
result_2=model_2.predict(x_test)

print("No round values")
print(mean_squared_error(y_test, result_2, squared=False))

print("Round values")
mean_squared_error(y_test, np.round(result_2), squared=False)

No round values
0.8574181383666432
Round values


0.8750149045437963

# Save MODEL and Files

In [172]:
save_path_1='model_test_1.h5'
model.save(save_path_1)

save_path_2='model_test_2.h5'
model_2.save(save_path_2)


In [175]:
df_result=y_test_df.copy()
df_result["result"]=np.round(result)
df_result.to_csv('result_model_1.csv',index=False)

In [180]:
df_result_2=y_test_df.copy()
df_result_2["result"]=np.round(result_2)
df_result_2.to_csv('result_model_2.csv',index=False)
#df_result_2.to_csv('/content/drive/MyDrive/GIT/result_model_2.csv',index=False)


In [178]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive
